## 使用数据增强的特征提取

扩展 conv_base 模型，然后在输入数据上端到端地运行模型。

注意：只在有 GPU 的情况下才能尝试运行。它在 CPU 上是绝对难以运行的。

In [1]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
# 在卷积基上添加一个密集连接分类器
from keras import models
from keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [3]:
#冻结网络
conv_base.trainable = False

In [ ]:
# 利用冻结的卷积基端到端地训练模型
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import os
base_dir = './data/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=50)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
100/100 [==============================] - 681s 7s/step - loss: 0.5919 - acc: 0.7085 - val_loss: 0.4407 - val_acc: 0.8380
Epoch 2/30
100/100 [==============================] - 662s 7s/step - loss: 0.4891 - acc: 0.7870 - val_loss: 0.3777 - val_acc: 0.8540
Epoch 3/30
100/100 [==============================] - 658s 7s/step - loss: 0.4285 - acc: 0.8190 - val_loss: 0.3297 - val_acc: 0.8680
Epoch 4/30
100/100 [==============================] - 846s 8s/step - loss: 0.4086 - acc: 0.8195 - val_loss: 0.3101 - val_acc: 0.8750
Epoch 5/30
100/100 [==============================] - 1317s 13s/step - loss: 0.3943 - acc: 0.8220 - val_loss: 0.3117 - val_acc: 0.8710
Epoch 6/30
 99/100 [============================>.] - ETA: 5s - loss: 0.3703 - acc: 0.8449 

In [ ]:
# 绘制训练过程中的损失曲线和精度曲线
import matplotlib.pyplot as plt 
%matplotlib inline

acc = history.history['acc'] 
val_acc = history.history['val_acc'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss'] 

epochs = range(1, len(acc) + 1) 

plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy') 
plt.legend() 

plt.figure() 

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss') 
plt.legend() 

plt.show()